## Univariate CNN Models

In [1]:
import numpy as np

from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.layers.convolutional import MaxPooling1D, Conv1D

Using TensorFlow backend.


In [2]:
data = [10, 20, 30, 40, 50, 60, 70, 80, 90]

In [3]:
def split_data(data, look_back: int = 3):
    x, y = list(), list()
    
    end_start = look_back
    for i in range(len(data)):
        if end_start > len(data) - 1:
            break
        
        x.append(data[i:end_start])
        y.append(data[end_start])
        
        end_start += 1
    
    return np.array(x), np.array(y)


In [4]:
look_back = 3

In [5]:
x, y = split_data(data, look_back)

print(x.shape, y.shape)

(6, 3) (6,)


In [6]:
x = x.reshape(x.shape[0], x.shape[1], 1)
y = y.reshape(-1, 1)

print(x.shape, y.shape)

(6, 3, 1) (6, 1)


In [7]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation="relu", input_shape=(x.shape[1], x.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())

model.add(Dense(50, activation="relu"))
model.add(Dense(1))
model.compile(optimizer="adam", loss="mse")
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 2, 64)             192       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                3250      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 3,493
Trainable params: 3,493
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.fit(x, y, epochs=1000, verbose=1)

Epoch 1/1000
6/6 [==============================] - 1s 125ms/step - loss: 4764.2954
Epoch 2/1000
6/6 [==============================] - 0s 4ms/step - loss: 4591.4316
Epoch 3/1000
6/6 [==============================] - 0s 4ms/step - loss: 4423.9004
Epoch 4/1000
6/6 [==============================] - 0s 3ms/step - loss: 4265.1885
Epoch 5/1000
6/6 [==============================] - 0s 2ms/step - loss: 4122.1509
Epoch 6/1000
6/6 [==============================] - 0s 952us/step - loss: 3983.6370
Epoch 7/1000
6/6 [==============================] - 0s 1ms/step - loss: 3851.3655
Epoch 8/1000
6/6 [==============================] - 0s 1ms/step - loss: 3722.0183
Epoch 9/1000
6/6 [==============================] - 0s 4ms/step - loss: 3598.2830
Epoch 10/1000
6/6 [==============================] - 0s 664us/step - loss: 3474.4265
Epoch 11/1000
6/6 [==============================] - 0s 4ms/step - loss: 3353.7571
Epoch 12/1000
6/6 [==============================] - 0s 2ms/step - loss: 3235.0039
Epoch 1

In [9]:
x_input = np.array([70, 80, 90])
x_input = x_input.reshape(1, x_input.shape[0], 1)
x_input.shape

(1, 3, 1)

In [10]:
yhat = model.predict(x_input, verbose=1)
yhat

1/1 [==============================] - 0s 131ms/step


array([[101.27165]], dtype=float32)

## Multivariate CNN Models

In [12]:
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [13]:
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))
dataset

array([[ 10,  15,  25],
       [ 20,  25,  45],
       [ 30,  35,  65],
       [ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

In [17]:
def split_data(data, look_back: int = 3):
    x, y = list(), list()
    
    for i in range(len(data)):
        end_start = i + look_back
        
        if end_start > len(data):
            break
        
        x.append(data[i:end_start, :-1])
        y.append(data[end_start - 1, -1])
        
        end_start += 1
    
    return np.array(x), np.array(y)

In [20]:
x, y = split_data(dataset, look_back)
y = y.reshape(-1, 1)
print(x.shape, y.shape)

(7, 3, 2) (7, 1)


In [21]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation="relu", input_shape=(x.shape[1], x.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())

model.add(Dense(50, activation="relu"))
model.add(Dense(y.shape[1]))
model.compile(optimizer="adam", loss="mse")
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 2, 64)             320       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                3250      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 3,621
Trainable params: 3,621
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(x, y, epochs=1000, verbose=1)

Epoch 1/1000
7/7 [==============================] - 1s 109ms/step - loss: 19654.4414
Epoch 2/1000
7/7 [==============================] - 0s 1ms/step - loss: 18805.2695
Epoch 3/1000
7/7 [==============================] - 0s 2ms/step - loss: 18097.1152
Epoch 4/1000
7/7 [==============================] - 0s 2ms/step - loss: 17367.3496
Epoch 5/1000
7/7 [==============================] - 0s 2ms/step - loss: 16587.7383
Epoch 6/1000
7/7 [==============================] - 0s 974us/step - loss: 15863.0410
Epoch 7/1000
7/7 [==============================] - 0s 859us/step - loss: 15148.6631
Epoch 8/1000
7/7 [==============================] - 0s 1ms/step - loss: 14447.1621
Epoch 9/1000
7/7 [==============================] - 0s 2ms/step - loss: 13759.5264
Epoch 10/1000
7/7 [==============================] - 0s 935us/step - loss: 13083.4863
Epoch 11/1000
7/7 [==============================] - 0s 1ms/step - loss: 12415.5176
Epoch 12/1000
7/7 [==============================] - 0s 1ms/step - loss: 1176

In [27]:
# demonstrate prediction
x_input = np.array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, x_input.shape[0], x_input.shape[1]))
x_input.shape

(1, 3, 2)

In [28]:
yhat = model.predict(x_input, verbose=1)
yhat

1/1 [==============================] - 0s 129ms/step


array([[207.07118]], dtype=float32)